# Example usage

To use `extrucal` in a project:

In [1]:
import extrucal

print(extrucal.__version__)

0.1.0


In [2]:
import pandas as pd
import numpy as np

In [ ]:
def output_per_rpm(size, pitch, depth):
    """
    Calculates the volumetric throughput per rpm given the screw size, 
    the channel depth of metering channel, and screw pitch
    ----------
    size  : float
            screw size
    pitch : flot
            screw pitch
    depth : float
            the channel depth of metering section

    Returns
    -------
    float : volumetric throughput per rpm

    Examples
    --------
    >>> output_per_rpm(size=100, pitch=10, depth=5)
    """




In [ ]:
def output_table(size, pitch, min_depth, max_depth, delta_depth, min_rpm, max_rpm, delta_rpm):
    """
    Calculates the volumetric throughput per rpm given the screw size, 
    the channel depth of metering channel, and screw pitch
    ----------
    size        : float
                  screw size
    pitch       : flot
                  screw pitch
    min_depth   : float
                  minimum depth for calculation
    max_depth   : float
                  maximum depth for calculation
    delta_depth : float
                  the amount of increment in depth for calculation
    min_rpm     : float
                  minimum screw RPM for calculation
    max_rpm     : float
                  maximum screw RPM for calculation
    delta_rpm   : float
                  the amount of increment in RPM for calculation

    Returns
    -------
    table : pandas.DataFrame
            dataframe containing the throughput as a function of channel depth and screw RPM

    Examples
    --------
    >>> output_table(size=100, pitch=10, min_depth=3, max_depth=7, delta_depth=1, min_rpm=10, max_rpm=50, delta_rpm=10)
    """

In [ ]:
def output_plot(size, pitch, min_depth, max_depth, delta_depth, min_rpm, max_rpm, delta_rpm):
    """
    Calculates the volumetric throughput per rpm given the screw size, 
    the channel depth of metering channel, and screw pitch
    ----------
    size        : float
                  screw size
    pitch       : flot
                  screw pitch
    min_depth   : float
                  minimum depth for calculation
    max_depth   : float
                  maximum depth for calculation
    delta_depth : float
                  the amount of increment in depth for calculation
    min_rpm     : float
                  minimum screw RPM for calculation
    max_rpm     : float
                  maximum screw RPM for calculation
    delta_rpm   : float
                  the amount of increment in RPM for calculation

    Returns
    -------
    plot : altair.Chart object
           a line chart showing the throughput as a function of channel depth and screw RPM

    Examples
    --------
    >>> output_plot(size=100, pitch=10, min_depth=3, max_depth=7, delta_depth=1, min_rpm=10, max_rpm=50, delta_rpm=10)
    """